In [ ]:
from family_network_model import *
from matplotlib import pyplot as plt
import networkx as nx
import ast
import pickle

## Run Model

In [ ]:
# get data

# marriage distance data
with open('smdistances.txt') as f:
    sm_dis = f.readline()
# San Marino marriage distances w/o infinite distances
smdistances = ast.literal_eval(sm_dis)

# number of infinite marriage distances 
inf = 4316

# number of children data
with open('smchildren.txt') as f:
    sm_child = f.readline()
# San Marino number of children
smchildren = ast.literal_eval(sm_child)

# probability of marriage-- marriage_edges*2 / total_nodes
P = (8168*2)/28586

# probability of non-connected marriage-- not_children / all_married nodes (not_children are nodes that are married but are not children)
NCP = 2672/15696

In [ ]:
# initialize parameters
n = 99   # n+1 = number of people in network
gen = 3  # use small number of generations for testing
name = 'test' # change every time you run model if you want to save that specific output
marriage_dist = smdistances
children_dist = smchildren
p = P
ncp = NCP
infdis = round((inf/8168 - (NCP/2))*8168)


# run model
G, D, unions, children, infs = human_family_network(n, gen, marriage_dist, p, ncp, infdis, children_dist, name)

# visualize resulting network
nx.draw(G, with_labels=True, font_weight='bold')

## Analyze Model

In [ ]:
def model_marriage_dis(unions, D):
    """Find distances of marriages formed in modeled network--used to compare to original data of distances
    Parameters:
                unions (list): list of all unions in modeled network
                D (array): matrix of distances between all nodes in modeled network
    Returns:
                distances (list): list of distances to marriage in modeled network
    
    """

    distances = []

    for pair in unions:
        p1, p2 = pair
        distances.append(D[p1][p2])
        
    return distances

### Visualize distribution of union distances at each generation

In [ ]:
for i in range(0, gen+2):
    print("Generation :", i)

    # read in files
    gpickle_file = nx.read_gpickle("test_G{}.gpickle".format(i))
    D_file = np.load("test_D{}.npy".format(i))
    with open ('test_U{}'.format(i), 'rb') as fup:
        unions_file = pickle.load(fup)
    with open('test_C{}'.format(i), 'rb') as fcp:
        children_file = pickle.load(fcp)

    # assign names 
    G = gpickle_file
    unions = unions_file
    D = D_file
    children = children_file

    # network size
    print("Number of nodes: ", len(G.nodes))

    # visualize distances
    distances = model_marriage_dis(unions, D)
    d = np.array(distances)
    mask1 = d < 100
    mask2 = d >= 100
    print("Number of marriages: ", len(unions))
    print("Number of infinite marriages: ", len(d[mask2]))
    
    plt.hist(d[mask1])
    plt.title("Generation {}".format(i))
    plt.show()

plt.hist(smdistances, color='r')
plt.title("San Marino Family Network")
plt.show()

### Visualize distribution of number of children at each generation

In [ ]:
for i in range(1, gen+2):
    print("Generation :", i)

    # read in files
    gpickle_file = nx.read_gpickle("test_G{}.gpickle".format(i))
    D_file = np.load("test_D{}.npy".format(i))
    with open ('test_U{}'.format(i), 'rb') as fup:
        unions_file = pickle.load(fup)
    with open('test_C{}'.format(i), 'rb') as fcp:
        children_file = pickle.load(fcp)

    # assign names 
    G = gpickle_file
    unions = unions_file
    D = D_file
    children = children_file

    # network size
    print("Number of nodes: ", len(G.nodes))

    plt.hist(children, color='g', bins=6)
    plt.title("Generation {}".format(i))
    plt.show()

plt.hist(smchildren, color='r', bins=6)
plt.title("San Marino Family Network")
plt.show()

### Visualize largest connected component of network at each generation

In [ ]:
# Visualize largest connected component

for i in range(gen+2):
    print("Generation :", i)

    # read in file
    gpickle_file = nx.read_gpickle("test_G{}.gpickle".format(i))
    G = gpickle_file
    
    Gcc = sorted(nx.connected_components(G), key=len, reverse=True)
    G0 = G.subgraph(Gcc[0])
    nx.draw(G0, node_size=15)
    plt.show()